In [ ]:
# - Bibliotecas para manipulação e análise de dados
import pandas as pd
import numpy as np
import openpyxl

# - Bibliotecas para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# - Bibliotecas para ML
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# - Configurações de visualização
%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

print("Bibliotecas importadas com sucesso!")

In [ ]:
# Criando o DataFrame
df = pd.read_excel('teste_contabeis_2024.xlsx')

# Exibindo as primeiras linhas do DataFrame
df.head()

In [ ]:
# Mostrando a dimensão do DataFrame
print(f"O DataFrame possui {df.shape[0]} linhas e {df.shape[1]} colunas.")
print("--------------------------------")
# Informações técnicas do DataFrame
df.info()
print("--------------------------------")
# pegando colunas e colocando em dataset
dataset = df.columns.tolist()
print(dataset)

print("--------------------------------")
# Verificando contas contábeis a serem classificadas
num_classes = df['CONTA'].nunique()
print(f"Número de contas contábeis únicas: {num_classes}")
print("--------------------------------")

In [ ]:
# Montagem de grafico de barras para visualizar a distribuição das classes
print("Distribuição de lançamentos por conta contábil: ")
count_contas = df['CONTA'].value_counts()
print(count_contas)
print("--------------------------------")
# Plotando o gráfico de barras
plt.figure(figsize=(15, 8))
sns.barplot(x=count_contas.index, y=count_contas.values, palette="viridis")
plt.title("Distribuição de Lançamentos por Conta Contábil")
plt.xlabel("Conta Contábil")
plt.ylabel("Número de Lançamentos")
plt.xticks(rotation=90)
plt.show()


In [ ]:
# Tratamento de texto - descrição
import re
from nltk.corpus import stopwords

# função para limpar o texto
def clean_text(text):
    # transformar em minúsculas
    text = str(text).lower()
    # remover numeros
    text = re.sub(r'\d+', ' ', text)
    # remover pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', ' ', text)
    # remover espaços extras
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    # remover stopwords
    stopwords_pt = set(stopwords.words('portuguese'))
    # Separar as palavras
    words = text.split()
    # Remover as stopwords
    words_filtered = [word for word in words if word not in stopwords_pt and len(word) > 2]
    # Juntar as palavras novamente
    clean_text = ' '.join(words_filtered)
    return clean_text

print("Limpando os textos das descrições...")
# Aplicando a função de limpeza ao DataFrame
# Criando uma coluna com o texto limpo
df["DESCRIÇÃO_LIMPA"] = df["DESCRIÇÃO DO LANÇAMENTO"].apply(clean_text)
# Comparação
print("Comparação entre texto original e texto limpo:")
df[["DESCRIÇÃO DO LANÇAMENTO", "DESCRIÇÃO_LIMPA"]].head(10)

In [ ]:
# Análise de frequência das palavras

count_vectorizer = CountVectorizer()
# matriz de contagem de palavras
arr_count = count_vectorizer.fit_transform(df["DESCRIÇÃO_LIMPA"])
# Somando as ocorrências de cada palavra
sum_words = arr_count.sum(axis=0)
# Criando um dicionário de palavras e suas frequências
words_freq = [(word, sum_words[0, idx]) for word, idx in count_vectorizer.vocabulary_.items()]
# Ordenar lista da mais frequente para a menos frequente
words_freq_sorted = sorted(words_freq, key=lambda x: x[1], reverse=True)
# Criando DataFrame
df_words_freq = pd.DataFrame(words_freq_sorted, columns=['Palavra', 'Frequência'])

# Mostrando as palavras mais frequentes
print("Palavras mais frequentes nas descrições limpas:")
df_words_freq.head(50) 

In [ ]:
# Criando primeiro modelo de teste
X = df["DESCRIÇÃO_LIMPA"]
y = df["CONTA"]
# Divisão entre dados de treino e teste: 70% treino e 30% teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# Definir e criar pipeline
text_clf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])   
# Treinando o pipeline
text_clf_pipeline.fit(X_train, y_train)
print("Modelo treinado com sucesso!")    

In [ ]:
# Avaliando o modelo, criando matriz de confusão e relatório de classificação
# Usando o pipeline para fazer previsões
y_pred = text_clf_pipeline.predict(X_test)
# Imprimindo o relatório de classificação
print("Relatório de Classificação: ")
print(classification_report(y_test, y_pred))
# Gerar e visualizar matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
labels = sorted(y_test.unique())
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.ylabel('Conta Real')
plt.xlabel('Conta Prevista')
plt.title('Matriz de Confusão')
plt.show()